In [24]:
import pandas as pd

In [25]:
traits = pd.read_csv("../scripts/2d_traits_per_IDDate 1.csv")
traits.head()

,ID,Date,Variety,Ripe,Part-ripe,Unripe,Flower,Fruit-width,Fruit-height,Mass,Yield/plant,Crop-composition,Plant-height,ExG
0,AA11,20250506,Falco,0,0,10,5,[],[],[],0.000000,0.000000,221.0,37.211900
1,AA11,20250516,Falco,0,0,19,0,[],[],[],0.000000,0.000000,319.0,37.279159
2,AA11,20250518,Falco,0,0,20,4,[],[],[],0.000000,0.000000,349.0,35.802554
3,AA11,20250520,Falco,1,0,9,3,[59.0],[55.0],[109267.70790456783],109267.707905,0.076923,367.0,37.359857
4,AA11,20250522,Falco,0,0,10,0,[],[],[],0.000000,0.000000,407.0,33.605164


In [26]:
traits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3595 entries, 0 to 3594
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                3595 non-null   object 
 1   Date              3595 non-null   int64  
 2   Variety           3595 non-null   object 
 3   Ripe              3595 non-null   int64  
 4   Part-ripe         3595 non-null   int64  
 5   Unripe            3595 non-null   int64  
 6   Flower            3595 non-null   int64  
 7   Fruit-width       3595 non-null   object 
 8   Fruit-height      3595 non-null   object 
 9   Mass              3595 non-null   object 
 10  Yield/plant       3595 non-null   float64
 11  Crop-composition  3355 non-null   float64
 12  Plant-height      3595 non-null   float64
 13  ExG               3595 non-null   float64
dtypes: float64(4), int64(5), object(5)
memory usage: 393.3+ KB


In [27]:
import ast
import math
def parse_list_field(val):
    """
    Convert string like "[59.0, 40.0]" into Python list.
    Empty list "[]" stays as [].
    NaN/None stays as [].
    """
    if val is None or (isinstance(val, float) and math.isnan(val)):
        return []
    if isinstance(val, str):
        try:
            parsed = ast.literal_eval(val)
            if isinstance(parsed, list):
                return parsed
            return []
        except Exception:
            return []
    if isinstance(val, list):
        return val
    return []

traits['Fruit-width'] = traits['Fruit-width'].apply(parse_list_field)
traits['Fruit-height '] = traits['Fruit-height'].apply(parse_list_field)
traits['Mass'] = traits['Mass'].apply(parse_list_field)

# Validate if there is row with unmatching number of fruits and ripe 

In [28]:
traits['Fruit-width_count'] = traits['Fruit-width'].apply(len)
traits['Fruit-height_count'] = traits['Fruit-height '].apply(len)
traits['Mass_count'] = traits['Mass'].apply(len)

In [29]:
invalid_traits = traits[(traits['Fruit-width_count'] != traits['Fruit-height_count']) | (traits['Fruit-width_count'] != traits['Mass_count']) | (traits['Fruit-width_count'] != traits['Ripe'])]
invalid_traits.head()

,ID,Date,Variety,Ripe,Part-ripe,Unripe,Flower,Fruit-width,Fruit-height,Mass,Yield/plant,Crop-composition,Plant-height,ExG,Fruit-height,Fruit-width_count,Fruit-height_count,Mass_count


# Manipulate dataset to seed data images (files) to PostgreSQL database

### 2D

In [30]:
traits_2d_images = traits[['ID', 'Date']].copy()
traits_2d_images.loc[:, 'FilePath'] = '2025/08/EX01/strawberries.webp'
traits_2d_images.loc[:, 'FileType'] = 'TWO_D'

In [31]:
traits_2d_images.to_csv("../scripts/traits_2d_images.csv", index=False)

### 3D

In [32]:
traits_2d_images = traits[['ID', 'Date']].copy()
traits_2d_images.loc[:, 'FilePath'] = '2025/08/EX01/dbscan_cleaned.ply'
traits_2d_images.loc[:, 'FileType'] = 'THREE_D'

In [33]:
traits_2d_images.to_csv("../scripts/traits_3d_images.csv", index=False)

# Select pot_ids which have at least one ripe fruit

In [36]:
has_ripe_fruit = traits[traits["Ripe"] > 0]["ID"].unique().tolist()
exclude = [
    "AA7",
    "AA21",
    "AA29",
    "AB24",
    "AD8",
    "AE9",
    "AF8",
    "AG13",
    "AH24",
    "AL32",
    "AM33",
    "AO35",
    "BO26",
    "BP15",
    "BP21",
    "BP11",
    "BP21",
    "BD17",
    "BB29",
    "BB31",
    "BO16",
    "BB19",
    "BO28",
    "BB17",
    "BB15",
    "BB27",
    "AH34",
    "AE27",
    "AG29",
    "AC17",
    "AA3",
    "AB14",
]
rest = [id_ for id_ in has_ripe_fruit if id_ not in exclude]
print("IDs with ripe fruit:", len(has_ripe_fruit))
res = rest[:167]
res.sort()
import json
print(json.dumps(res))

IDs with ripe fruit: 222
["AA11", "AA13", "AA15", "AA17", "AA19", "AA23", "AA25", "AA27", "AA31", "AA33", "AA35", "AA9", "AB12", "AB16", "AB18", "AB2", "AB20", "AB22", "AB28", "AB30", "AB32", "AB34", "AB4", "AB8", "AC11", "AC13", "AC15", "AC21", "AC23", "AC25", "AC27", "AC29", "AC3", "AC31", "AC33", "AC35", "AC5", "AC7", "AC9", "AD10", "AD12", "AD14", "AD16", "AD20", "AD22", "AD24", "AD26", "AD28", "AD30", "AD32", "AD34", "AD4", "AD6", "AE11", "AE13", "AE15", "AE17", "AE19", "AE21", "AE25", "AE29", "AE31", "AE33", "AE35", "AE5", "AE7", "AF10", "AF12", "AF14", "AF16", "AF18", "AF2", "AF20", "AF22", "AF24", "AF26", "AF28", "AF30", "AF32", "AF34", "AF4", "AF6", "AG11", "AG15", "AG19", "AG21", "AG23", "AG25", "AG27", "AG31", "AG33", "AG35", "AG7", "AG9", "AH10", "AH12", "AH14", "AH16", "AH18", "AH2", "AH20", "AH26", "AH28", "AH30", "AH32", "AH4", "AH6", "AH8", "AI11", "AI13", "AI15", "AI17", "AI19", "AI21", "AI23", "AI27", "AI29", "AI3", "AI31", "AI33", "AI35", "AI5", "AI7", "AI9", "AJ12",